In [3]:
import numpy as np
import pandas as pd

In [4]:
train_df = pd.read_csv("../fileFromDb/inputFile.csv")

In [5]:
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10682 entries, 0 to 10681
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10682 non-null  object
 1   Date_of_Journey  10682 non-null  object
 2   Source           10682 non-null  object
 3   Destination      10682 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10682 non-null  object
 6   Arrival_Time     10682 non-null  object
 7   Duration         10682 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10682 non-null  object
 10  Price            10682 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.1+ KB


## Removing unneccessary columns

**1. There is no need of Route column since the Total_Stops column is present**

**2. There is need to keep the columns for departure time and arrival times since the dataframe contains the Duration column**

In [7]:
train_df.drop(columns=['Route','Dep_Time','Arrival_Time'],axis=1,inplace=True)

In [8]:
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,4h 45m,1 stop,No info,13302


**Changing the datatype of the column Date_of_Journey to datetime**

In [9]:
train_df['Date_of_Journey'] = pd.to_datetime(train_df['Date_of_Journey'],format="%d/%m/%Y")

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10682 entries, 0 to 10681
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          10682 non-null  object        
 1   Date_of_Journey  10682 non-null  datetime64[ns]
 2   Source           10682 non-null  object        
 3   Destination      10682 non-null  object        
 4   Duration         10682 non-null  object        
 5   Total_Stops      10682 non-null  object        
 6   Additional_Info  10682 non-null  object        
 7   Price            10682 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 667.8+ KB


**Creating columns names Day_of_Journey, Month_of_Journey and Year_of_Journey from the column Year_of_Journey and dropping the original columns**

In [11]:
train_df['Day_of_Journey'] = train_df['Date_of_Journey'].dt.day
train_df['Month_of_Journey'] = train_df['Date_of_Journey'].dt.month
train_df['Year_of_Journey'] = train_df['Date_of_Journey'].dt.year

In [12]:
train_df.drop(columns=['Date_of_Journey'],axis=1,inplace=True)

In [13]:
train_df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey
0,IndiGo,Banglore,New Delhi,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,7h 25m,2 stops,No info,7662,1,5,2019
2,Jet Airways,Delhi,Cochin,19h,2 stops,No info,13882,9,6,2019
3,IndiGo,Kolkata,Banglore,5h 25m,1 stop,No info,6218,12,5,2019
4,IndiGo,Banglore,New Delhi,4h 45m,1 stop,No info,13302,1,3,2019


**Creating a new column called Flight_Duration from the column called Duration which will contain the flight duration in minutes**

In [14]:
import re

In [15]:
pattern1 = re.compile(r"(\d+)(h|m)(\s)(\d*)(h|m)*")
pattern2 = re.compile(r"(\s*)(\d+)(h)")
pattern3 = re.compile(r"(\s*)(\d+)(m)")

In [16]:
min_lst = []
for i in range(train_df.shape[0]):
    if 'h' in train_df.iloc[i,3] and 'm' in train_df.iloc[i,3]:
        matchobj = re.match(pattern1,train_df.iloc[i,3])
        hour = int(matchobj.group(1))
        minute = int(matchobj.group(4))
        total_min = 60*hour + minute
        min_lst.append(total_min)
    elif 'h' in train_df.iloc[i,3] and 'm' not in train_df.iloc[i,3]:
        matchobj = re.match(pattern2,train_df.iloc[i,3])
        hour = int(matchobj.group(2))
        min_lst.append(60*hour)
    elif 'h' not in train_df.iloc[i,3] and 'm' in train_df.iloc[i,3]:
        matchobj = re.match(pattern3,train_df.iloc[i,3])
        minute = int(matchobj.group(2))
        min_lst.append(minute)
    else:
        min_lst.append(train_df.iloc[i,3])

In [17]:
train_values = pd.Series(min_lst)
train_df.insert(loc=3, column="Flight_Duration", value=train_values)
train_df.drop(columns=['Duration'],axis=1,inplace=True)

In [18]:
train_df.head()

,Airline,Source,Destination,Flight_Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey
0,IndiGo,Banglore,New Delhi,170,non-stop,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,445,2 stops,No info,7662,1,5,2019
2,Jet Airways,Delhi,Cochin,1140,2 stops,No info,13882,9,6,2019
3,IndiGo,Kolkata,Banglore,325,1 stop,No info,6218,12,5,2019
4,IndiGo,Banglore,New Delhi,285,1 stop,No info,13302,1,3,2019


**Changing the Total_Stops column into integer datatype**

In [19]:
train_df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [20]:
train_df = train_df.drop(9039,axis=0)

In [21]:
train_df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [22]:
dict = {'non-stop': 0,'1 stop': 1, '2 stops': 2, '3 stops': 3, '4 stops': 4}

In [23]:
train_df['Total_Stops'] = train_df['Total_Stops'].map(dict)

In [24]:
train_df.head()

,Airline,Source,Destination,Flight_Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey
0,IndiGo,Banglore,New Delhi,170,0,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,445,2,No info,7662,1,5,2019
2,Jet Airways,Delhi,Cochin,1140,2,No info,13882,9,6,2019
3,IndiGo,Kolkata,Banglore,325,1,No info,6218,12,5,2019
4,IndiGo,Banglore,New Delhi,285,1,No info,13302,1,3,2019


In [25]:
a= train_df.pop("Price")
train_df.insert(9,"Price",a)

In [26]:
train_df.head()

,Airline,Source,Destination,Flight_Duration,Total_Stops,Additional_Info,Day_of_Journey,Month_of_Journey,Year_of_Journey,Price
0,IndiGo,Banglore,New Delhi,170,0,No info,24,3,2019,3897
1,Air India,Kolkata,Banglore,445,2,No info,1,5,2019,7662
2,Jet Airways,Delhi,Cochin,1140,2,No info,9,6,2019,13882
3,IndiGo,Kolkata,Banglore,325,1,No info,12,5,2019,6218
4,IndiGo,Banglore,New Delhi,285,1,No info,1,3,2019,13302


In [27]:
preprocess1 = train_df.to_csv("../DataPreprocessingPart2/preprocess1.csv",header=True,index=None)